# Execution of the Bell circuit

The "Hello World!" of Quantum Computing is the generation of the 2-qubit Bell state.

## Creating the EPR/Bell state 

Let us start with the circuit:

In [1]:
from mpqp import QCircuit
from mpqp.gates import H, CNOT

In [2]:
circuit = QCircuit([H(0), CNOT(0,1)], label="Bell state")
print(circuit)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


## Run the circuit on a local device

We can execute a circuit on a local simulator by calling the function ``run`` and precising the device.

In [3]:
from mpqp.execution import run, ATOSDevice

When no measure is added to the circuit, running the circuit will consist in
extracting the state-vector at the output of the circuit.

> ⚠ This feature is not supported on all backends

In [4]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.amplitudes)
print(result.probabilities)

Result: Bell state, ATOSDevice, MYQLM_PYLINALG
 State vector: [0.70711, 0, 0, 0.70711]
 Probabilities: [0.5, 0, 0, 0.5]
 Number of qubits: 2
[0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
[0.5 0.  0.  0.5]


We can also add to the circuit a ``BasisMeasure``, consisting in sample the state in a given basis. By default, the basis is the computational one.

In [5]:
from mpqp.measures import BasisMeasure

We precise which qubits we can to measure by inputting a list of indices, and precising the number of shots. When ``shots=0``, we end up in the same case as before, a statevector simulation.

In [6]:
circuit.add(BasisMeasure([0,1], shots=0))
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)

Result: Bell state, ATOSDevice, MYQLM_PYLINALG
 State vector: [0.70711, 0, 0, 0.70711]
 Probabilities: [0.5, 0, 0, 0.5]
 Number of qubits: 2


When we precise a number of shots, the circuit will be sampled and the core of
the `Result` will be a list of `Sample`. A precising the counts for each state
of the basis.

In [7]:
circuit = circuit.without_measurements()
circuit.add(BasisMeasure([0,1], shots=1024))

In [8]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.samples)
print(result.counts)
print(result.probabilities)

Result: Bell state, ATOSDevice, MYQLM_PYLINALG
 Counts: [476, 0, 0, 548]
 Probabilities: [0.46484, 0, 0, 0.53516]
 Samples:
  State: 00, Index: 0, Count: 476, Probability: 0.4648438
  State: 11, Index: 3, Count: 548, Probability: 0.5351562
 Error: 0.015593944424785589
[Sample(2, index=0, count=476, probability=0.46484375), Sample(2, index=3, count=548, probability=0.53515625)]
[476, 0, 0, 548]
[0.46484375 0.         0.         0.53515625]


## Run the circuit on multiple devices

By using the same function ``run`` we can execute the circuit on several simulators at the time. One just has to give a list of devices instead of a single device.

In [9]:
from mpqp.execution import IBMDevice, AWSDevice, GOOGLEDevice

In [10]:
results = run(circuit, [ATOSDevice.MYQLM_PYLINALG, IBMDevice.AER_SIMULATOR, AWSDevice.BRAKET_LOCAL_SIMULATOR, GOOGLEDevice.CIRQ_LOCAL_SIMULATOR])
print(results)

BatchResult: 4 results
Result: Bell state, ATOSDevice, MYQLM_PYLINALG
 Counts: [516, 0, 0, 508]
 Probabilities: [0.50391, 0, 0, 0.49609]
 Samples:
  State: 00, Index: 0, Count: 516, Probability: 0.5039062
  State: 11, Index: 3, Count: 508, Probability: 0.4960938
 Error: 0.01563215790957881
Result: Bell state, IBMDevice, AER_SIMULATOR
 Counts: [497, 0, 0, 527]
 Probabilities: [0.48535, 0, 0, 0.51465]
 Samples:
  State: 00, Index: 0, Count: 497, Probability: 0.4853516
  State: 11, Index: 3, Count: 527, Probability: 0.5146484
 Error: None
Result: Bell state, AWSDevice, BRAKET_LOCAL_SIMULATOR
 Counts: [522, 0, 0, 502]
 Probabilities: [0.50977, 0, 0, 0.49023]
 Samples:
  State: 00, Index: 0, Count: 522, Probability: 0.5097656
  State: 11, Index: 3, Count: 502, Probability: 0.4902344
 Error: None
Result: Bell state, GOOGLEDevice, CIRQ_LOCAL_SIMULATOR
 Counts: [515, 0, 0, 509]
 Probabilities: [0.50293, 0, 0, 0.49707]
 Samples:
  State: 00, Index: 0, Count: 515, Probability: 0.5029297
  State:

In [11]:
print(results[0])

Result: Bell state, ATOSDevice, MYQLM_PYLINALG
 Counts: [516, 0, 0, 508]
 Probabilities: [0.50391, 0, 0, 0.49609]
 Samples:
  State: 00, Index: 0, Count: 516, Probability: 0.5039062
  State: 11, Index: 3, Count: 508, Probability: 0.4960938
 Error: 0.01563215790957881


## Run or submit the circuit on a remote device

To execute the circuit on remote device, one can use the exact same process as
with local devices. A call of the function `run` on a remote device will launch
the job and wait until it finished before returning the result. One or
several devices can still be given in parameter.

In [12]:
result = run(circuit, IBMDevice.IBM_LEAST_BUSY)
print(result)

Result: Bell state, IBMDevice, IBM_KYIV
 Counts: [489, 13, 11, 511]
 Probabilities: [0.47754, 0.0127, 0.01074, 0.49902]
 Samples:
  State: 00, Index: 0, Count: 489, Probability: 0.4775391
  State: 01, Index: 1, Count: 13, Probability: 0.0126953
  State: 10, Index: 2, Count: 11, Probability: 0.0107422
  State: 11, Index: 3, Count: 511, Probability: 0.4990234
 Error: None


However, it is also possible to asynchronously submit the job way using the `submit` function.

In [13]:
from mpqp.execution import submit

By submitting the circuit to a remote device, we retrieve the id of the job
attributed by the provider, as well as the corresponding MPQP job.

The MPQP job object contains additional information, such as the `status` of the job.

In [14]:
job_id, job = submit(circuit, IBMDevice.IBM_LEAST_BUSY)
print(job_id)

czpnk3mkzhn0008bpnsg


Once the computation is done, we use the function ``get_remote_result`` for
retrieving the result. 

If the job is not completed, the function will wait (blocking) until it is done.

In [15]:
from mpqp.execution import get_remote_result

In [16]:
result = get_remote_result(job_id, IBMDevice.IBM_LEAST_BUSY)
print(result)

Result: IBMDevice, IBM_KYIV
 Counts: [485, 2, 10, 527]
 Probabilities: [0.47363, 0.00195, 0.00977, 0.51465]
 Samples:
  State: 00, Index: 0, Count: 485, Probability: 0.4736328
  State: 01, Index: 1, Count: 2, Probability: 0.0019531
  State: 10, Index: 2, Count: 10, Probability: 0.0097656
  State: 11, Index: 3, Count: 527, Probability: 0.5146484
 Error: None
